In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
iitm1=pd.read_json('iitm1.json')
iitm2=pd.read_json('iitm2.json')

In [3]:
tweets_df = pd.concat([iitm1,iitm2], axis=0)
tweets_df = tweets_df.loc[:, ['text' , 'username' ,'tweet_id', 'timestamp' ,'hashtags' , 'screen_name','retweets','likes']]
tweets_df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes
0,Interesting visit to Sotacarbo centre of excel...,India in Italy,1138170993191600128,2019-06-10 19:48:05,[],IndiainItaly,13,78
1,@iitmadras This research can save $300 billio...,Hari Desai,1138080195263361024,2019-06-10 13:47:17,[],HariDesai12,0,0
2,@iitmadras You are reliable technology instit...,Hari Desai,1138079376455548928,2019-06-10 13:44:02,[],HariDesai12,0,0
3,#iitmadras hosts 2nd edition of #carbonzerocha...,"IIT Madras, Alumni",1138061888523317248,2019-06-10 12:34:33,"[iitmadras, carbonzerochallenge]",iitm_alumni,0,0
4,Centre for Social Innovation and Entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,[FacultyDevelopmentProgram],iitmadras,0,13
5,.@iitmadras hosts 2nd edition of Carbon Zero C...,Education Times,1138027583700783104,2019-06-10 10:18:14,"[sustainablesolutions, energy]",educationtimes,2,12
6,IIT Madras hosted valedictory function; Comple...,StudyBuzz,1138026135340457984,2019-06-10 10:12:28,[],studybuzztweet,0,1
7,Over 350 students from rural pockets of TN wit...,IITM Incubation Cell,1138025319988809728,2019-06-10 10:09:14,[StartupJobUtsav],IITMIC,5,8
8,A panel discussion on the DRAFT NEW EDUCATION ...,evmalert,1138023350851100672,2019-06-10 10:01:24,[],evmalert,0,25
9,"Ah, you let the cat out of the bag and Yes,",Sairam Radhakrishnan,1137999507810082816,2019-06-10 08:26:40,[],sairamrkn,0,1


In [4]:
tweets_df.reset_index(drop=True, inplace=True)

In [5]:
#check how many tweets were from one of the university handles
tweets_df = tweets_df[~tweets_df.index.duplicated()]
uni_tweets = tweets_df.loc[tweets_df['screen_name'].isin(['iitmadras','ecelliitm','vandemataramiit','iGEM_IITM','iitm_respark','domsiitmadras','emliitm','iitmbt','CFI_IITM','iitmfoundation'])]

In [6]:
uni_tweets['uni_handle'] = 1

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
tweets_df = tweets_df[~tweets_df.index.duplicated()]
tweets_df['uni_handle'] = uni_tweets['uni_handle']
iitm = tweets_df.combine_first(tweets_df)
iitm

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
0,Interesting visit to Sotacarbo centre of excel...,India in Italy,1138170993191600128,2019-06-10 19:48:05,[],IndiainItaly,13,78,NaN
1,@iitmadras This research can save $300 billio...,Hari Desai,1138080195263361024,2019-06-10 13:47:17,[],HariDesai12,0,0,NaN
2,@iitmadras You are reliable technology instit...,Hari Desai,1138079376455548928,2019-06-10 13:44:02,[],HariDesai12,0,0,NaN
3,#iitmadras hosts 2nd edition of #carbonzerocha...,"IIT Madras, Alumni",1138061888523317248,2019-06-10 12:34:33,"[iitmadras, carbonzerochallenge]",iitm_alumni,0,0,NaN
4,Centre for Social Innovation and Entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,[FacultyDevelopmentProgram],iitmadras,0,13,1.0
5,.@iitmadras hosts 2nd edition of Carbon Zero C...,Education Times,1138027583700783104,2019-06-10 10:18:14,"[sustainablesolutions, energy]",educationtimes,2,12,NaN
6,IIT Madras hosted valedictory function; Comple...,StudyBuzz,1138026135340457984,2019-06-10 10:12:28,[],studybuzztweet,0,1,NaN
7,Over 350 students from rural pockets of TN wit...,IITM Incubation Cell,1138025319988809728,2019-06-10 10:09:14,[StartupJobUtsav],IITMIC,5,8,NaN
8,A panel discussion on the DRAFT NEW EDUCATION ...,evmalert,1138023350851100672,2019-06-10 10:01:24,[],evmalert,0,25,NaN
9,"Ah, you let the cat out of the bag and Yes,",Sairam Radhakrishnan,1137999507810082816,2019-06-10 08:26:40,[],sairamrkn,0,1,NaN


In [8]:
iitm['uni_handle'] = iitm['uni_handle'].replace(np.nan, 0)
iitm[iitm['uni_handle'] == 1]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
4,Centre for Social Innovation and Entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,[FacultyDevelopmentProgram],iitmadras,0,13,1.0
14,@iitmadras and @VirtusaCorp organized the #Car...,IIT Madras,1137972974135332864,2019-06-10 06:41:14,"[CarbonZeroChallenge, prototypes, GrandFinale,...",iitmadras,2,15,1.0
23,Centre for Social Innovation and Entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,[FacultyDevelopmentProgram],iitmadras,0,13,1.0
33,@iitmadras and @VirtusaCorp organized the #Car...,IIT Madras,1137972974135332864,2019-06-10 06:41:14,"[CarbonZeroChallenge, prototypes, GrandFinale,...",iitmadras,2,15,1.0
43,Path that leads to beautiful destinations - Ro...,IIT Madras,1137652598725414912,2019-06-09 09:28:10,[],iitmadras,23,177,1.0
56,@iitmadras on Jun 5th hosted the valedictory f...,IIT Madras,1137260824236421120,2019-06-08 07:31:24,[RSIC],iitmadras,3,22,1.0
66,@iitmadras tops #Outlook ranking of India's to...,E-Cell IIT Madras,1137031963301142528,2019-06-07 16:21:59,"[Outlook, Engineering]",ecelliitm,0,0,1.0
69,@iitmadras and @VirtusaCorp hosted the 2nd ed...,IIT Madras,1137001872235978752,2019-06-07 14:22:25,[CarbonZeroChallenge],iitmadras,3,23,1.0
77,All set for the #GrandFinaleExhibition and #Aw...,IIT Madras,1136877359884201984,2019-06-07 06:07:39,"[GrandFinaleExhibition, AwardCeremony, carbonz...",iitmadras,3,45,1.0
88,Congratulations @iitmadras !! https://twitter....,E-Cell IIT Madras,1136636522558398464,2019-06-06 14:10:39,[],ecelliitm,0,0,1.0


In [9]:
#Finding sentiment of the tweets using VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
iitm['compound'] = [analyzer.polarity_scores(v)['compound'] for v in iitm['text']]
iitm

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound
0,Interesting visit to Sotacarbo centre of excel...,India in Italy,1138170993191600128,2019-06-10 19:48:05,[],IndiainItaly,13,78,0.0,0.6908
1,@iitmadras This research can save $300 billio...,Hari Desai,1138080195263361024,2019-06-10 13:47:17,[],HariDesai12,0,0,0.0,0.7508
2,@iitmadras You are reliable technology instit...,Hari Desai,1138079376455548928,2019-06-10 13:44:02,[],HariDesai12,0,0,0.0,0.7430
3,#iitmadras hosts 2nd edition of #carbonzerocha...,"IIT Madras, Alumni",1138061888523317248,2019-06-10 12:34:33,"[iitmadras, carbonzerochallenge]",iitm_alumni,0,0,0.0,-0.4019
4,Centre for Social Innovation and Entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,[FacultyDevelopmentProgram],iitmadras,0,13,1.0,0.6124
5,.@iitmadras hosts 2nd edition of Carbon Zero C...,Education Times,1138027583700783104,2019-06-10 10:18:14,"[sustainablesolutions, energy]",educationtimes,2,12,0.0,0.4939
6,IIT Madras hosted valedictory function; Comple...,StudyBuzz,1138026135340457984,2019-06-10 10:12:28,[],studybuzztweet,0,1,0.0,0.0000
7,Over 350 students from rural pockets of TN wit...,IITM Incubation Cell,1138025319988809728,2019-06-10 10:09:14,[StartupJobUtsav],IITMIC,5,8,0.0,0.4215
8,A panel discussion on the DRAFT NEW EDUCATION ...,evmalert,1138023350851100672,2019-06-10 10:01:24,[],evmalert,0,25,0.0,0.0000
9,"Ah, you let the cat out of the bag and Yes,",Sairam Radhakrishnan,1137999507810082816,2019-06-10 08:26:40,[],sairamrkn,0,1,0.0,0.4019


In [10]:
#classifying sentiments as positive,negative and neutral
def sentiment(compound):
    res = ""
    if(compound >= 0.05):
        res="Positive"
    elif(compound <= - 0.05):
        res="Negative"
    else:
        res="Neutral"
    return res

iitm['Sentiment'] = iitm['compound'].apply(lambda x: sentiment(x))

In [11]:
iitm['score'] = pd.cut(iitm['compound'], bins=5, labels=[1, 2, 3, 4, 5])
iitm

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,Interesting visit to Sotacarbo centre of excel...,India in Italy,1138170993191600128,2019-06-10 19:48:05,[],IndiainItaly,13,78,0.0,0.6908,Positive,5
1,@iitmadras This research can save $300 billio...,Hari Desai,1138080195263361024,2019-06-10 13:47:17,[],HariDesai12,0,0,0.0,0.7508,Positive,5
2,@iitmadras You are reliable technology instit...,Hari Desai,1138079376455548928,2019-06-10 13:44:02,[],HariDesai12,0,0,0.0,0.7430,Positive,5
3,#iitmadras hosts 2nd edition of #carbonzerocha...,"IIT Madras, Alumni",1138061888523317248,2019-06-10 12:34:33,"[iitmadras, carbonzerochallenge]",iitm_alumni,0,0,0.0,-0.4019,Negative,2
4,Centre for Social Innovation and Entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,[FacultyDevelopmentProgram],iitmadras,0,13,1.0,0.6124,Positive,5
5,.@iitmadras hosts 2nd edition of Carbon Zero C...,Education Times,1138027583700783104,2019-06-10 10:18:14,"[sustainablesolutions, energy]",educationtimes,2,12,0.0,0.4939,Positive,4
6,IIT Madras hosted valedictory function; Comple...,StudyBuzz,1138026135340457984,2019-06-10 10:12:28,[],studybuzztweet,0,1,0.0,0.0000,Neutral,3
7,Over 350 students from rural pockets of TN wit...,IITM Incubation Cell,1138025319988809728,2019-06-10 10:09:14,[StartupJobUtsav],IITMIC,5,8,0.0,0.4215,Positive,4
8,A panel discussion on the DRAFT NEW EDUCATION ...,evmalert,1138023350851100672,2019-06-10 10:01:24,[],evmalert,0,25,0.0,0.0000,Neutral,3
9,"Ah, you let the cat out of the bag and Yes,",Sairam Radhakrishnan,1137999507810082816,2019-06-10 08:26:40,[],sairamrkn,0,1,0.0,0.4019,Positive,4


In [12]:
iitm.to_csv('../../Datasets/Twitter/Datasets with likes/final_iitm_withlikes.csv')